In [ ]:
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import movingpandas as mpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi
# import torch

import scipy
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
import scipy.stats

In [ ]:
# Set the filename
# filename = '../data/SharkArray-2020-05-21-thru-05-28.csv'
# filename = '../data/2020-21-max-3-min-run.csv'
# filename = '../data/2020-13-141-long-5-min-run.csv'
filename = '../data/2020-21-477-long-15-min-run.csv'
# filename = '../data/2020-20-05-21.csv'
# filename = '../data/2020-20-full.csv'

# Load shark positions data into a GeoDataFrame
shark_gdf = pd.read_csv(filename)
shark_gdf['t'] = pd.to_datetime(shark_gdf['DATETIME'])
shark_gdf['geometry'] = gpd.points_from_xy(shark_gdf.LON, shark_gdf.LAT)
shark_gdf = gpd.GeoDataFrame(shark_gdf)
shark_gdf = shark_gdf.set_crs('EPSG:4326')
shark_gdf = shark_gdf.set_index('t').tz_localize(None)
shark_gdf.head()

filename = os.path.splitext(os.path.basename(filename))[0]
# filename += '-v2'

In [ ]:
output = '../graphs/{}'.format(filename)

In [ ]:
print('The dataset contains', shark_gdf.shape[0], 'rows and', shark_gdf.shape[1], 'columns.')
print('The column names are:', list(shark_gdf.columns.values))
print('The unique transmitter names are:', shark_gdf['TRANSMITTER'].unique())

In [ ]:
# Create separate trajectories for each shark based on their transmitter ID
traj_collection = mpd.TrajectoryCollection(shark_gdf, 'TRANSMITTER')
traj = traj_collection.trajectories[0]
print(traj_collection, traj)

In [ ]:
# Add a timedelta column which is the time between the previous position and the current position
n = traj.df.shape[0]
timedeltas = [timedelta()] + [traj.df.index[i] - traj.df.index[i - 1] for i in range(1, n)]
traj.df['TIMEDELTA'] = timedeltas

In [ ]:
# Add velocities and headings to each trajectory
traj.add_speed()
traj.add_direction()

In [ ]:
# Compute turning angles
def bound_angle_diff(theta_diff):
    return ((theta_diff - 180) % 360) - 180

n = traj.df.shape[0]
turning_angles = [0] + [bound_angle_diff(traj.df['direction'][i + 1] - traj.df['direction'][i]) for i in range(1, n - 1)] + [0]
traj.df['turning_angle'] = turning_angles

In [ ]:
traj.plot(linestyle='None')
plt.title('Longest 3-min run trajectory')
plt.savefig('{}/{}-traj'.format(output, filename), bbox_inches="tight")

In [ ]:
# Create a random generator
rng = np.random.default_rng()

In [ ]:
# Compute turning angles
def wrap_to_pi(theta):
    return ((theta - np.pi) % (2 * np.pi)) - np.pi

turning_angles = wrap_to_pi(np.radians(np.array(traj.df['turning_angle'])))
# turning_angles = tfd.VonMises(loc=-1, concentration=0).sample([207]).numpy()
print(turning_angles)
plt.hist(turning_angles, bins=np.linspace(-np.pi, np.pi, 30))
plt.title('Turning angle histogram')
plt.xlabel('angle (rad)')
plt.ylabel('counts')
plt.savefig('{}/{}-turning-angle-hist'.format(output, filename), bbox_inches="tight")

In [ ]:
pos_angles = [angle for angle in turning_angles if angle > 0]
neg_angles = [angle for angle in turning_angles if angle <= 0]
plt.hist(pos_angles, bins=np.linspace(-np.pi, np.pi, 30), alpha=0.8, label='positive angles')
plt.hist(neg_angles, bins=np.linspace(-np.pi, np.pi, 30), alpha=0.8, label='negative angles')
plt.title('Turning angle histogram')
plt.xlabel('angle (rad)')
plt.ylabel('counts')
plt.legend()

In [ ]:
# Plot some example von Mises distributions
num = 1001
locs = [2, 2, 0, 0]
cons = [0, 2, 1, 4]
x = np.linspace(-np.pi, np.pi, num).reshape(num, 1)
y = tfd.VonMises(loc=locs, concentration=cons).prob(x).numpy()
for i in range(y.shape[1]):
    plt.plot(x[:, 0], y[:, i], label='loc={:.2f}, concentration={:.2f}'.format(locs[i], cons[i]))
plt.gca().set_ylim((0, 1))
plt.title('Von Mises examples')
plt.xlabel('angle (rad)')
plt.ylabel('probability density')
plt.legend()
plt.savefig('../animations/von-mises-examples', bbox_inches="tight")
plt.show()

In [ ]:
# We assume 2 states
num_states = 2

# Randomly initialize the initial state distriubtion as well as the transition probabilities
initial_probs = tf.Variable(scipy.special.softmax(rng.random([num_states])), name='initial_probs')
transition_probs = tf.Variable(scipy.special.softmax(rng.random([num_states, num_states]), axis=1), name='transition_probs')

print("Initial state probs:\n{}".format(initial_probs))
print("Transition matrix:\n{}".format(transition_probs))

In [ ]:
# Initialize locations and concentrations of Von Mises distributions for turning angles
vm_locs = tf.Variable(np.zeros(num_states))
vm_cons = tf.Variable(np.zeros(num_states))

print('von Mises locations:\n{}'.format(vm_locs))
print('von Mises concentrations:\n{}'.format(vm_cons))

In [ ]:
# Create HMM
hmm = tfd.HiddenMarkovModel(
    initial_distribution = tfd.Categorical(probs=initial_probs),
    transition_distribution = tfd.Categorical(probs=transition_probs),
    observation_distribution = tfd.VonMises(loc=vm_locs, concentration=vm_cons),
    num_steps = len(turning_angles)
)

In [ ]:
# Define a loss function
def log_prob():
    return tf.reduce_logsumexp(hmm.log_prob(turning_angles))

# Define an optimizer to perform back propagation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

# Make sure probabilities sum to 1
def normalize_probs(probs):
    abs_probs = tf.math.abs(probs)
    if len(probs.shape) > 1:
        sums = tf.reshape(tf.reduce_sum(abs_probs, axis=1), [probs.shape[0], 1])
    else:
        sums = tf.reduce_sum(abs_probs)
    return abs_probs / sums

def wrap_to_pi(A):
    return ((A - np.pi) % (2 * np.pi) - np.pi)

# Run a step of the optimizer
@tf.function(autograph=False)
def train_op():
    with tf.GradientTape() as tape:
        neg_log_prob = -log_prob()
    vars = [initial_probs, transition_probs, vm_locs, vm_cons]
    grads = tape.gradient(neg_log_prob, vars)
    optimizer.apply_gradients(zip(grads, vars))
    initial_probs.assign(normalize_probs(initial_probs))
    transition_probs.assign(normalize_probs(transition_probs))
    vm_locs.assign(wrap_to_pi(vm_locs))
    vm_cons.assign(tf.math.abs(vm_cons))
    return neg_log_prob, *vars

In [ ]:
# Train on the observations
loss_history = []
for step in range(201):
    loss, ip, tp, vl, vc = [t.numpy() for t in train_op()]
    loss_history.append(loss)
    if step % 20 == 0:
        print("step {}: log prob {}\nInitial probs: {}\nTransition probs:\n{}\nVon Mises locs: {}\nVon Mises cons: {}\n".format(step, -loss, ip, tp, vl, vc))

In [ ]:
plt.plot(loss_history)
plt.savefig('{}/{}-univariate-loss-hist'.format(output, filename), bbox_inches="tight")

In [ ]:
# Plot von Mises distributions
num = 1001
x = np.linspace(-np.pi, np.pi, num).reshape(num, 1)
y = hmm.observation_distribution.prob(x).numpy()
for i in range(y.shape[1]):
    plt.plot(x[:, 0], y[:, i], label='state {}, loc={:.2f}, concentration={:.2f}'.format(i, vm_locs[i], vm_cons[i]))
plt.gca().set_ylim((0, 1))
plt.title('Emission probability distributions')
plt.xlabel('angle (rad)')
plt.ylabel('probability density')
plt.legend()
plt.savefig('{}/{}-univariate-emission-dists'.format(output, filename), bbox_inches="tight")
plt.show()

In [ ]:
posterior_dists = hmm.posterior_marginals(turning_angles)
posterior_probs = posterior_dists.probs_parameter().numpy()

In [ ]:
def plot_state_posterior(ax, state_posterior_probs, observed_data, title, label='turning angles', ylabel='angle (rad)'):
    ln1 = ax.plot(state_posterior_probs, c='blue', lw=3, label='p(state | angles)')
    ax.set_ylim(0., 1.1)
    ax.set_ylabel('posterior probability')
    ax2 = ax.twinx()
    ln2 = ax2.plot(observed_data, c='black', alpha=0.3, label=label)
    ax2.set_title(title)
    ax2.set_ylabel(ylabel)
    lns = ln1 + ln2
    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc=4)
    ax.grid(True, color='white')
    ax2.grid(False)

fig, axs = plt.subplots(num_states, 1, figsize=(7, 5 * num_states))
for state in range(num_states):
    plot_state_posterior(axs[state], posterior_probs[:, state], turning_angles, 'state {} (mean turning angle {:.2f} rad)'.format(state, vm_locs[state]))

plt.savefig('{}/{}-univariate-obs-posterior-probs'.format(output, filename), bbox_inches="tight")

In [ ]:
x = [point.coords[0][0] for point in traj.df['geometry']]
y = [point.coords[0][1] for point in traj.df['geometry']]
fig, ax = plt.subplots(figsize=(10, 10))
cmaplist = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'][:num_states]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, num_states)
color = np.argmax(posterior_probs, axis=1)
sc = ax.scatter(x, y, c=color, cmap=cmap)
traj.df['state'] = color
traj.plot(ax=ax, marker='o', column='state', cmap=cmap)
ticks = np.array(list(range(num_states)))
tick_labels = ['state {}'.format(i) for i in range(num_states)]
cbar = plt.colorbar(sc, fraction=0.03)
cbar.set_ticks(ticks)
cbar.set_ticklabels(tick_labels)
plt.title('Trajectory with states')
plt.savefig('{}/{}-univariate-obs-traj-with-states'.format(output, filename), bbox_inches="tight")

In [ ]:
np.sum(np.argmax(posterior_probs, axis=1))

In [ ]:
# Save to csv with states
traj.df.to_csv('../data/{}-{}-states.csv'.format(filename, num_states), index=False)

In [ ]:
speeds = np.array(traj.df['speed'])
print(speeds, len(speeds))
plt.hist(speeds, bins=np.linspace(0, np.ceil(np.max(speeds)), 30))
plt.title('Speed histogram')
plt.xlabel('speed (m/s)')
plt.ylabel('counts')
plt.savefig('{}/{}-speed-hist'.format(output, filename), bbox_inches="tight")

In [ ]:
# We assume 2 states
num_states = 2

# Randomly initialize the initial state distriubtion as well as the transition probabilities
initial_probs = tf.Variable(scipy.special.softmax(rng.random([num_states])), name='initial_probs')
transition_probs = tf.Variable(scipy.special.softmax(rng.random([num_states, num_states]), axis=1), name='transition_probs')

print("Initial state probs:\n{}".format(initial_probs))
print("Transition matrix:\n{}".format(transition_probs))

In [ ]:
# Plot some example gamma distributions
num = 1001
shapes = [1, 1, 1, 2, 2, 2]
rates = [2, 1, 0.5, 2, 1, 0.5]
x = np.linspace(0, 6, num).reshape(num, 1)
y = tfd.Gamma(concentration=shapes, rate=rates).prob(x).numpy()
for i in range(y.shape[1]):
    plt.plot(x[:, 0], y[:, i], label='shape={:.2f}, rate={:.2f}'.format(shapes[i], rates[i]))
plt.gca().set_xlim((0, 6))
plt.gca().set_ylim((0, np.max(y)))
plt.title('Gamma examples')
plt.xlabel('speed (m/s)')
plt.ylabel('probability density')
plt.legend()
plt.savefig('../animations/gamma-examples', bbox_inches="tight")
plt.show()

In [ ]:
# Initialize locations and concentrations of Von Mises distributions for turning angles
vm_locs = tf.Variable(np.zeros(num_states))
vm_cons = tf.Variable(np.zeros(num_states))
# vm_locs = tf.Variable(rng.random(num_states))
# vm_cons = tf.Variable(rng.random(num_states))

# Initialize shapes and rates of Gamma distributions for step length
gamma_shapes = tf.Variable(np.ones(num_states))
gamma_rates = tf.Variable(np.ones(num_states))
# gamma_shapes = tf.Variable(rng.random(num_states))
# gamma_rates = tf.Variable(rng.random(num_states))

# joint_dists = tfd.JointDistributionSequential([
#     tfd.VonMises(loc=vm_locs, concentration=vm_cons),
#     tfd.Gamma(concentration=gamma_shapes, rate=gamma_rates)
# ])

joint_dists = tfd.Blockwise([
    tfd.VonMises(loc=vm_locs, concentration=vm_cons),
    tfd.Gamma(concentration=gamma_shapes, rate=gamma_rates)
])

print('von Mises locations:\n{}'.format(vm_locs))
print('von Mises concentrations:\n{}'.format(vm_cons))
print('Gamma shapes:\n{}'.format(gamma_shapes))
print('Gamma rates:\n{}'.format(gamma_rates))
print(joint_dists)

In [ ]:
# Create HMM
hmm2 = tfd.HiddenMarkovModel(
    initial_distribution = tfd.Categorical(probs=initial_probs),
    transition_distribution = tfd.Categorical(probs=transition_probs),
    observation_distribution = joint_dists,
    num_steps = len(turning_angles)
)

In [ ]:
hmm2.log_prob(np.array([turning_angles, speeds]).T)

In [ ]:
# Define a loss function
def log_prob():
    return tf.reduce_logsumexp(hmm2.log_prob(np.array([turning_angles, speeds]).T))

# Define an optimizer to perform back propagation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

# Make sure probabilities sum to 1
def normalize_probs(probs):
    abs_probs = tf.math.abs(probs)
    if len(probs.shape) > 1:
        sums = tf.reshape(tf.reduce_sum(abs_probs, axis=1), [probs.shape[0], 1])
    else:
        sums = tf.reduce_sum(abs_probs)
    return abs_probs / sums

def wrap_to_pi(A):
    return ((A - np.pi) % (2 * np.pi) - np.pi)

# Run a step of the optimizer
@tf.function(autograph=False)
def train_op():
    with tf.GradientTape() as tape:
        neg_log_prob = -log_prob()
    vars = [initial_probs, transition_probs, vm_locs, vm_cons, gamma_shapes, gamma_rates]
    grads = tape.gradient(neg_log_prob, vars)
    optimizer.apply_gradients(zip(grads, vars))
    initial_probs.assign(normalize_probs(initial_probs))
    transition_probs.assign(normalize_probs(transition_probs))
    vm_locs.assign(wrap_to_pi(vm_locs))
    vm_cons.assign(tf.math.abs(vm_cons))
    gamma_shapes.assign(tf.math.abs(gamma_shapes))
    gamma_rates.assign(tf.math.abs(gamma_rates))
    return (neg_log_prob, *vars), grads

In [ ]:
# Train on the observations
loss_history = []
for step in range(1000):
    ts, grads = train_op()
    loss, ip, tp, vl, vc, gs, gr = [t.numpy() for t in ts]
    loss_history.append(loss)
    if step % 100 == 0:
        print("step {}: log prob {}\nInitial probs: {}\nTransition probs:\n{}\nVon Mises locs: {}\nVon Mises cons: {}\nGamma shapes: {}\nGamma rates: {}\n".format(step, -loss, ip, tp, vl, vc, gs, gr))

In [ ]:
plt.plot(loss_history)
plt.savefig('{}/{}-bivariate-loss-hist'.format(output, filename), bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot observation distributions
num = 1001
x = np.linspace(-np.pi, np.pi, num).reshape(num, 1)
for (j, (obs_dist, ax)) in enumerate(zip(hmm2.observation_distribution.distributions, axs)):
    y = obs_dist.prob(x).numpy()
    for i in range(y.shape[1]):
        if j == 0:
            label = 'state {}, loc={:.2f}, concentration={:.2f}'.format(i, vm_locs[i], vm_cons[i])
            title = 'Turning angle distributions'
        else:
            label = 'state {}, mean={:.2f}, mode={:.2f}'.format(i, gamma_shapes[i] / gamma_rates[i], (gamma_shapes[i] - 1) / gamma_rates[i])
            title = 'Speed distributions'
        ax.plot(x[:, 0], y[:, i], label=label)
        ax.set_title(title)
        ax.legend(loc='upper right')
plt.savefig('{}/{}-bivariate-emission-dists'.format(output, filename), bbox_inches="tight")
plt.show()

In [ ]:
posterior_dists = hmm2.posterior_marginals(np.array([turning_angles, speeds]).T)
posterior_probs = posterior_dists.probs_parameter().numpy()

In [ ]:
fig, axs = plt.subplots(num_states, 2, figsize=(14, 5 * num_states))
for state, ax_row in enumerate(axs):
    for i, ax in enumerate(ax_row):
        if i == 0:
            plot_state_posterior(ax, posterior_probs[:, state], turning_angles, 'state {} (mean turning angle {:.2f} rad)'.format(state, vm_locs[state]))
        else:
            plot_state_posterior(ax, posterior_probs[:, state], speeds, 'state {} (mean speed {:.2f} m/s)'.format(state, gamma_shapes[state] / gamma_rates[state]), label='speed', ylabel='speed (m/s)')
#         ax.set_xlim((0, 100))
plt.savefig('{}/{}-bivariate-obs-posterior-probs'.format(output, filename), bbox_inches="tight")    

In [ ]:
x = [point.coords[0][0] for point in traj.df['geometry']]
y = [point.coords[0][1] for point in traj.df['geometry']]
fig, ax = plt.subplots(figsize=(10, 10))
cmaplist = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'][:num_states]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, num_states)
color = np.argmax(posterior_probs, axis=1)
sc = ax.scatter(x, y, c=color, cmap=cmap)
traj.df['state'] = color
traj.plot(ax=ax, marker='o', column='state', cmap=cmap)
ticks = np.array(list(range(num_states)))
tick_labels = ['state {}'.format(i) for i in range(num_states)]
cbar = plt.colorbar(sc, fraction=0.03)
cbar.set_ticks(ticks)
cbar.set_ticklabels(tick_labels)
plt.title('Trajectory with states')
plt.savefig('{}/{}-bivariate-obs-traj-with-states'.format(output, filename), bbox_inches="tight")

In [ ]:
# Save to csv with states
traj.df.to_csv('../data/{}-{}-states-with-speeds.csv'.format(filename, num_states), index=False)

In [ ]:
def get_sample_traj(sampled_obs, traj):
    '''
    Converts a sample from an HMM to a trajectory
    '''
    # Bound angles to [-pi, pi]
    def wrap_to_pi(theta):
        return ((theta - 180) % 360) - 180
    
    # Convert sampled observations to x and y positions
    sampled_x = [0]
    sampled_y = [0]
    sampled_angle = [0]
    sampled_turning_angle = [sampled_obs[0][0]]
    sampled_speed = [sampled_obs[1][0]]
    times = np.array([dt.total_seconds() for dt in traj.df['TIMEDELTA']])
    for i, (dt, (turning_angle, speed)) in enumerate(zip(times, sampled_obs)):
        prev_x = sampled_x[i]
        prev_y = sampled_y[i]
        prev_angle = sampled_angle[i]
        step_length = dt * speed
        angle = wrap_to_pi(prev_angle + turning_angle)
        x = step_length * np.cos(angle) + prev_x
        y = step_length * np.sin(angle) + prev_y
        sampled_x.append(x)
        sampled_y.append(y)
        sampled_angle.append(angle)
        sampled_turning_angle.append(turning_angle)
        sampled_speed.append(speed)
    
    # Align the samples' centroids and make them the same scale
    sampled_x = np.array(sampled_x)
    sampled_y = np.array(sampled_y)
    avg_x = np.average(sampled_x)
    avg_y = np.average(sampled_y)
    sampled_x -= avg_x
    sampled_y -= avg_y
    actual_x = []
    actual_y = []
    for point in traj.df['geometry']:
        x, y = point.coords[0]
        actual_x.append(x)
        actual_y.append(y)
    actual_x = np.array(actual_x)
    actual_y = np.array(actual_y)
    avg_x = np.average(actual_x)
    avg_y = np.average(actual_y)
    actual_range_x = np.max(actual_x) - np.min(actual_x)
    actual_range_y = np.max(actual_y) - np.min(actual_y)
    sampled_range_x = np.max(sampled_x) - np.min(sampled_x)
    sampled_range_y = np.max(sampled_y) - np.min(sampled_y)
    sampled_x *= actual_range_x / sampled_range_x
    sampled_y *= actual_range_y / sampled_range_y
    sampled_x += avg_x
    sampled_y += avg_y
    data = np.array([sampled_x, sampled_y, sampled_angle, sampled_turning_angle, sampled_speed])
    sample_traj = pd.DataFrame(data.T[1:], columns=['LON', 'LAT', 'direction', 'turning_angle', 'speed'])
    sample_traj['t'] = traj.df.index
    sample_traj['geometry'] = gpd.points_from_xy(sample_traj.LON, sample_traj.LAT)
    sample_traj = gpd.GeoDataFrame(sample_traj)
    sample_traj = sample_traj.set_crs('EPSG:4326')
    sample_traj = sample_traj.set_index('t').tz_localize(None)
    return mpd.Trajectory(sample_traj, traj.id + '-sampled')

In [ ]:
# Sample the HMM to get a path
sampled_obs = hmm2.sample().numpy()
sample_traj = get_sample_traj(sampled_obs, traj)

In [ ]:
print(sample_traj)
print(traj)

In [ ]:
fig, ax = plt.subplots()
traj.plot(ax=ax, label='actual path', color='#1f77b4')
sample_traj.plot(ax=ax, label='sampled path', color='#ff7f0e')
plt.legend()
plt.savefig('{}/{}-sampled-traj'.format(output, filename), bbox_inches="tight")

In [ ]:
plt.plot(traj.df['DEPTH'])

In [ ]:
observations = []
num_missing = 0
for idx, row in traj.df.iterrows():
    # Append missing observations for minutes without an observation
    for i in range(max(int(row['TIMEDELTA'].total_seconds() / 60) - 1, 0)):
        num_missing += 1
        observations.append([1, 1.0, 1.0])
    observations.append([0, np.radians(row['turning_angle']), row['speed']])
observations = np.array(observations)
p_missing = tf.constant(num_missing / len(observations), dtype=tf.float32)
print(p_missing, num_missing, len(observations))

In [ ]:
# We assume 2 states
num_states = 2

# Randomly initialize the initial state distribution as well as the transition probabilities
initial_probs = tf.Variable(scipy.special.softmax(rng.random([num_states])), name='initial_probs', dtype=tf.float32)
transition_probs = tf.Variable(scipy.special.softmax(rng.random([num_states, num_states]), axis=1), name='transition_probs', dtype=tf.float32)

print("Initial state probs:\n{}".format(initial_probs))
print("Transition matrix:\n{}".format(transition_probs))

In [ ]:
# Initialize locations and concentrations of Von Mises distributions for turning angles
vm_locs = tf.Variable(np.zeros(num_states), dtype=tf.float32)
vm_cons = tf.Variable(np.zeros(num_states), dtype=tf.float32)
# vm_locs = tf.Variable(rng.random(num_states))
# vm_cons = tf.Variable(rng.random(num_states))

# Initialize shapes and rates of Gamma distributions for step length
gamma_shapes = tf.Variable(np.ones(num_states), dtype=tf.float32)
gamma_rates = tf.Variable(np.ones(num_states), dtype=tf.float32)
# gamma_shapes = tf.Variable(rng.random(num_states))
# gamma_rates = tf.Variable(rng.random(num_states))

mixed_dists = tfd.Mixture(
    cat=tfd.Categorical(probs=[[1 - p_missing, p_missing]] * num_states),
    components=[
        tfd.Blockwise([
            tfd.Categorical(probs=[[1.0, 0.0]] * num_states),
            tfd.VonMises(loc=vm_locs, concentration=vm_cons),
            tfd.Gamma(concentration=gamma_shapes, rate=gamma_rates)
        ], dtype_override=tf.float32),
        tfd.Independent(
            tfd.Deterministic(loc=tf.zeros([num_states, 3]) + 1),
            reinterpreted_batch_ndims=1)
    ]
)

print('von Mises locations:\n{}'.format(vm_locs))
print('von Mises concentrations:\n{}'.format(vm_cons))
print('Gamma shapes:\n{}'.format(gamma_shapes))
print('Gamma rates:\n{}'.format(gamma_rates))
print(mixed_dists)

In [ ]:
# Create HMM
hmm3 = tfd.HiddenMarkovModel(
    initial_distribution = tfd.Categorical(probs=initial_probs),
    transition_distribution = tfd.Categorical(probs=transition_probs),
    observation_distribution = mixed_dists,
    num_steps = len(observations)
)

In [ ]:
# Define a loss function
def log_prob():
    return hmm3.log_prob(observations)

# Define an optimizer to perform back propagation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

# Make sure probabilities sum to 1
def normalize_probs(probs):
    abs_probs = tf.math.abs(probs)
    if len(probs.shape) > 1:
        sums = tf.reshape(tf.reduce_sum(abs_probs, axis=1), [probs.shape[0], 1])
    else:
        sums = tf.reduce_sum(abs_probs)
    return abs_probs / sums

def wrap_to_pi(A):
    return ((A - np.pi) % (2 * np.pi) - np.pi)

# Run a step of the optimizer
@tf.function(autograph=False)
def train_op():
    with tf.GradientTape() as tape:
        neg_log_prob = -log_prob()
    vars = [initial_probs, transition_probs, vm_locs, vm_cons, gamma_shapes, gamma_rates]
    grads = tape.gradient(neg_log_prob, vars)
    optimizer.apply_gradients(zip(grads, vars))
    initial_probs.assign(normalize_probs(initial_probs))
    transition_probs.assign(normalize_probs(transition_probs))
    vm_locs.assign(wrap_to_pi(vm_locs))
    vm_cons.assign(tf.math.abs(vm_cons))
    gamma_shapes.assign(tf.math.abs(gamma_shapes))
    gamma_rates.assign(tf.math.abs(gamma_rates))
    return (neg_log_prob, *vars), grads

In [ ]:
hmm3.log_prob(observations)

In [ ]:
# Train on the observations
loss_history = []
for step in range(500):
    ts, grads = train_op()
    loss, ip, tp, vl, vc, gs, gr = [t.numpy() for t in ts]
    loss_history.append(loss)
    if step % 50 == 0:
        print("step {}: log prob {}\nInitial probs: {}\nTransition probs:\n{}\nVon Mises locs: {}\nVon Mises cons: {}\nGamma shapes: {}\nGamma rates: {}\n".format(step, -loss, ip, tp, vl, vc, gs, gr))

In [ ]:
plt.savefig('{}/{}-missing-obs-loss-hist'.format(output, filename), bbox_inches="tight")
plt.plot(loss_history)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot observation distributions
num = 1001
x = np.linspace(-np.pi, np.pi, num).reshape(num, 1)
for (j, (obs_dist, ax)) in enumerate(zip([tfd.VonMises(loc=vm_locs, concentration=vm_cons), tfd.Gamma(concentration=gamma_shapes, rate=gamma_rates)], axs)):
    y = obs_dist.prob(x).numpy()
    for i in range(y.shape[1]):
        if j == 0:
            label = 'state {}, loc={:.2f}, concentration={:.2f}'.format(i, vm_locs[i], vm_cons[i])
            title = 'Turning angle distributions'
        else:
            label = 'state {}, mean={:.2f}, mode={:.2f}'.format(i, gamma_shapes[i] / gamma_rates[i], (gamma_shapes[i] - 1) / gamma_rates[i])
            title = 'Speed distributions'
        ax.plot(x[:, 0], y[:, i], label=label)
        ax.set_title(title)
        ax.legend(loc='upper right')
plt.savefig('{}/{}-missing-obs-emission-dists'.format(output, filename), bbox_inches="tight")
plt.show()

In [ ]:
posterior_dists = hmm3.posterior_marginals(tf.constant(observations, dtype=tf.float32))
posterior_probs = posterior_dists.probs_parameter().numpy()

In [ ]:
hmm2_posterior_mode = hmm2.posterior_mode(np.array([turning_angles, speeds]).T)
hmm3_posterior_mode = hmm3.posterior_mode(tf.constant(observations, dtype=tf.float32))

In [ ]:
hmm3_posterior_mode_matched = []
posterior_probs_matched = []
for state, probs, obs in zip(hmm3_posterior_mode, posterior_probs, observations):
    if obs[0] == 0:
        hmm3_posterior_mode_matched.append(state)
        posterior_probs_matched.append(probs)
hmm3_posterior_mode_matched = np.array(hmm3_posterior_mode_matched)
posterior_probs_matched = np.array(posterior_probs_matched)

In [ ]:
fig, axs = plt.subplots(num_states, 2, figsize=(14, 5 * num_states))
for state, ax_row in enumerate(axs):
    for i, ax in enumerate(ax_row):
        if i == 0:
            plot_state_posterior(ax, posterior_probs_matched[:, state], turning_angles, 'state {} (mean turning angle {:.2f} rad)'.format(state, vm_locs[state]))
        else:
            plot_state_posterior(ax, posterior_probs_matched[:, state], speeds, 'state {} (mean speed {:.2f} m/s)'.format(state, gamma_shapes[state] / gamma_rates[state]), label='speed', ylabel='speed (m/s)')
plt.savefig('{}/{}-missing-obs-posterior-probs'.format(output, filename), bbox_inches="tight")

In [ ]:
x = [point.coords[0][0] for point in traj.df['geometry']]
y = [point.coords[0][1] for point in traj.df['geometry']]
fig, ax = plt.subplots(figsize=(10, 10))
cmaplist = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'][:num_states]
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, num_states)
color = np.argmax(posterior_probs_matched, axis=1)
sc = ax.scatter(x, y, c=color, cmap=cmap)
traj.df['state'] = color
traj.plot(ax=ax, marker='o', column='state', cmap=cmap)
ticks = np.array(list(range(num_states)))
tick_labels = ['state {}'.format(i) for i in range(num_states)]
cbar = plt.colorbar(sc, fraction=0.03)
cbar.set_ticks(ticks)
cbar.set_ticklabels(tick_labels)
plt.title('Trajectory with states')
plt.savefig('{}/{}-missing-obs-traj-with-states'.format(output, filename), bbox_inches="tight")